# Grey-scale input model

Cells below we're used for testing grey-scale input images

In [ ]:
# Imports
from __future__ import print_function, division

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import PIL.Image
import shutil
import time
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import zipfile

from azure.storage.blob import BlockBlobService
from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets

In [ ]:
# Define constants
TRAIN_LABEL_FILE = 'Data/train_labels.csv'
TRAIN_IMAGES_DIR = 'Data/train'
CHECKPOINT_DIR = 'Checkpoint_V1'
BLOB_ACCOUNT_NAME = 'jtmlblobdatasets'
BLOB_ACCOUNT_KEY = '' # Removed for privacy
BLOB_CONTAINER = 'histcancerdetectiondata'
BLOB_NAME = 'train.zip'
DATASET_ZIP = 'Data/data.zip'
USE_GPU = torch.cuda.is_available()

In [ ]:
# Download zipped dataset from Azure Blob Storage
if (not os.path.isfile(DATASET_ZIP)):
    training_labels = pd.read_csv(TRAIN_LABEL_FILE)
    blob_service = BlockBlobService(account_name = BLOB_ACCOUNT_NAME, account_key = BLOB_ACCOUNT_KEY)

    start_time = time.time()

    blob_service.get_blob_to_path(container_name = BLOB_CONTAINER, blob_name = BLOB_NAME, file_path = DATASET_ZIP)

    end_time = time.time()

    print('Time to download: {} mins', round((end_time - start_time)/ 60., 2))

In [ ]:
# Testing access to zip file
archive = zipfile.ZipFile(DATASET_ZIP, 'r')
img = Image.open(archive.open('000aa5d8f68dc1f45ebba53b8f159aae80e06072.tif'))
plt.imshow(img)

print(torch.cuda.is_available())

In [ ]:
# PCamDatasetFromZipfile Class
class PCamDatasetFromZipfile2(Dataset):
    """  Histopathologic Cancer Detection Dataset from Zip File       """
    """  Updated to apply different transforms to train/dev/test set  """
    def __init__(self, zip_file, csv_file, train_indices, train_transform, test_transform):
        self.archive = zipfile.ZipFile(zip_file, 'r')
        self.labels = pd.read_csv(csv_file)
        self.train_indices = train_indices
        self.train_transform = train_transform
        self.test_transform = test_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        file = self.labels.iloc[idx, 0] + '.tif'
        image = Image.open(self.archive.open(file))
        label = self.labels.iloc[idx, 1]
        label = int(label.reshape((1)))
        
        if (idx in self.train_indices):
            image = self.train_transform(image)
        else:
            image = self.test_transform(image)
            
        return file, image, label

In [ ]:
class PCamDatasetFromAnalysisData(Dataset):
    """Histopathologic Cancer Detection Dataset from Analysis Data"""
    def __init__(self, analysis_data, transform):
        self.data = analysis_data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file = self.data[idx]['file']
        label = self.data[idx]['y_true']
        image = self.data[idx]['image']
        return file, self.transform(image), int(label)

In [ ]:
# V2 -- +1 Conv/FC & Dropout & for B/W Images
class CNN_V2_BW(nn.Module):
    """Convolutional Neural Network"""
    def __init__(self, p = 0.5):
        super(CNN_V2_BW, self).__init__()
        # 1. Convolutional layers
        self.conv1 = nn.Conv2d( in_channels = 1
                               , out_channels = 16
                               , kernel_size = 3
                               , stride = 1
                               , padding = 1 )
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d( in_channels = 16
                               , out_channels = 32
                               , kernel_size = 3
                               , stride = 1
                               , padding = 1 )
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d( in_channels = 32
                               , out_channels = 64
                               , kernel_size = 3
                               , stride = 1
                               , padding = 1 )
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d( in_channels = 64
                               , out_channels = 128
                               , kernel_size = 3
                               , stride = 1
                               , padding = 1 )
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d( kernel_size = 2
                                 , stride = 2
                                 , padding = 0 )
        self.dropout = nn.Dropout(p = p)
        
        # 2. FC layers to final output
        self.fc1 = nn.Linear(in_features = 128*6*6, out_features = 512)
        self.fc_bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(in_features = 512, out_features = 256)
        self.fc_bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(in_features = 256, out_features = 128)
        self.fc_bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(in_features = 128, out_features = 1)

    def forward(self, x):
        # Apply 4x...
        # Convolution Layers, followed by Batch Normalizations, Maxpool, and ReLU
        x = self.bn1(self.conv1(x))                      # batch_size x 96 x 96 x 16
        x = self.pool(F.relu(x))                         # batch_size x 48 x 48 x 16
        x = self.bn2(self.conv2(x))                      # batch_size x 48 x 48 x 32
        x = self.pool(F.relu(x))                         # batch_size x 24 x 24 x 32
        x = self.bn3(self.conv3(x))                      # batch_size x 24 x 24 x 64
        x = self.pool(F.relu(x))                         # batch_size x 12 x 12 x 64
        x = self.bn4(self.conv4(x))                      # batch_size x 12 x 12 x 128
        x = self.pool(F.relu(x))                         # batch_size x  6 x  6 x 128
        
        # Flatten the output for each image
        x = x.view(-1, self.num_flat_features(x))        # batch_size x 6*6*128
        
        # Apply 4 FC Layers
        x = self.fc1(x)
        x = self.fc_bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = self.fc_bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        x = self.fc_bn3(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.fc4(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
# Helper functions
def show_image(image, label):
    """Show image with label"""
    print('Label: ' + str(label[0]))
    plt.show(image)

def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def training_accuracy(predicted, true, i, acc, tpr, tnr):
    """Taken from https://www.kaggle.com/krishanudb/cancer-detection-deep-learning-model-using-pytorch"""
    predicted = predicted.cpu()
    true = true.cpu()
    
    predicted = (sigmoid(predicted.data.numpy()) > 0.5)
    true = true.data.numpy()
    
    accuracy = np.sum(predicted == true) / true.shape[0]
    #true_positive_rate = np.sum((predicted == 1) * (true == 1)) / np.sum(true == 1)
    #true_negative_rate = np.sum((predicted == 0) * (true == 0)) / np.sum(true == 0)
    acc = acc * (i) / (i + 1) + accuracy / (i + 1)
    
    tpr = 0.0
    tnr = 0.0
    #tpr = tpr * (i) / (i + 1) + true_positive_rate / (i + 1)
    #tnr = tnr * (i) / (i + 1) + true_negative_rate / (i + 1)
    return acc, tpr, tnr

def dev_accuracy(predicted, target):
    """Taken from https://www.kaggle.com/krishanudb/cancer-detection-deep-learning-model-using-pytorch"""
    predicted = predicted.cpu()
    target = target.cpu()
    predicted = (sigmoid(predicted.data.numpy()) > 0.5)
    true = target.data.numpy()
    accuracy = np.sum(predicted == true) / true.shape[0]
    true_positive_rate = np.sum((predicted == 1) * (true == 1)) / np.sum(true == 1)
    true_negative_rate = np.sum((predicted == 0) * (true == 0)) / np.sum(true == 0)
    return accuracy, true_positive_rate, true_negative_rate

def train_dev_test_split_indices(dataset_size, train_split=0.8, dev_split=0.1, seed=30):
    """Given a dataset size, returns the train/dev/test split indices"""
    indices = list(range(dataset_size))
    train_split_end = int(np.floor(train_split * dataset_size))
    dev_split_end = int(np.floor(dev_split * dataset_size)) + train_split_end
    np.random.seed(seed)
    np.random.shuffle(indices)
    train_indices = indices[:train_split_end]
    dev_indices = indices[train_split_end:dev_split_end]
    test_indices = indices[dev_split_end:]
    return train_indices, dev_indices, test_indices

def train_dev_test_split_indices_from_dataset(dataset, train_split=0.8, dev_split=0.1, seed=30):
    """Given a dataset, returns the train/dev/test split indices"""
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    train_split_end = int(np.floor(train_split * dataset_size))
    dev_split_end = int(np.floor(dev_split * dataset_size)) + train_split_end
    np.random.seed(seed)
    np.random.shuffle(indices)
    train_indices = indices[:train_split_end]
    dev_indices = indices[train_split_end:dev_split_end]
    test_indices = indices[dev_split_end:]
    return train_indices, dev_indices, test_indices

def fetch_state(epoch, model, optimizer, dev_loss_min, dev_acc_max):
    """Returns the state dictionary for a model and optimizer"""
    state = {
        'epoch': epoch,
        'dev_loss_min': dev_loss_min,
        'dev_acc_max': dev_acc_max,
        'state_dict': model.state_dict(),
        'optim_dict': optimizer.state_dict()
    }
    return state

def save_checkpoint(state, is_best = False, checkpoint = CHECKPOINT_DIR):
    """Taken from CS230 PyTorch Code Examples"""
    """Saves model and training parameters at checkpoint + 'last.pth.tar'. If is_best==True, also saves
    checkpoint + 'best.pth.tar'

    Args:
        state: (dict) contains model's state_dict, may contain other keys such as epoch, optimizer state_dict
        is_best: (bool) True if it is the best model seen till now
        checkpoint: (string) folder where parameters are to be saved
    """
    filepath = os.path.join(checkpoint, 'last_v2_hybrid.pth.tar')
    if (not os.path.exists(checkpoint)):
        print("Checkpoint Directory does not exist! Making directory {}".format(checkpoint))
        os.mkdir(checkpoint)
    else:
        print("Checkpoint Directory exists! ")
    torch.save(state, filepath)
    if (is_best):
        shutil.copyfile(filepath, os.path.join(checkpoint, 'best_v2_hybrid.pth.tar'))
        
def load_checkpoint(model, optimizer = None, checkpoint = CHECKPOINT_DIR):
    """Taken from CS230 PyTorch Code Examples"""
    """Loads model parameters (state_dict) from file_path. If optimizer is provided, loads state_dict of
    optimizer assuming it is present in checkpoint.

    Args:
        checkpoint: (string) filename which needs to be loaded
        model: (torch.nn.Module) model for which the parameters are loaded
        optimizer: (torch.optim) optional: resume optimizer from checkpoint
    """
    if not os.path.exists(checkpoint):
        raise("File doesn't exist {}".format(checkpoint))
    checkpoint = torch.load(checkpoint)
    model.load_state_dict(checkpoint['state_dict'])

    if optimizer:
        optimizer.load_state_dict(checkpoint['optim_dict'])

    return checkpoint

In [ ]:
print(CNN_V2_BW())

In [ ]:
# Transformations
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels = 1),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels = 1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Parameters
batch_size = 16
num_workers = 0
num_epochs = 100
total_epochs = 0
early_stop_limit = 10
bad_epoch_count = 0
stop = False
train_loss_min = np.Inf
dev_loss_min = np.Inf
dev_acc_max = 0

In [ ]:
# Get dataset size to get train/dev/test indices
dataset_size = len(pd.read_csv(TRAIN_LABEL_FILE))
print(dataset_size)

# Create data indices for train/dev set split
train_indices, dev_indices, test_indices = train_dev_test_split_indices(dataset_size)

# Create DataSet
dataset = PCamDatasetFromZipfile2( zip_file = DATASET_ZIP
                             , csv_file = TRAIN_LABEL_FILE
                             , train_indices = train_indices
                             , train_transform = train_transforms
                             , test_transform = test_transforms )

train_sampler = SubsetRandomSampler(train_indices)   # 176,020 Images (Full) / [train_subset_size] Images (Subset)
dev_sampler   = SubsetRandomSampler(dev_indices)     # 22,002 Images (Full) / [dev_subset_size] Images (Subset)
test_sampler  = SubsetRandomSampler(test_indices)    # 22,003 Images (Full)

train_loader = DataLoader( dataset = dataset
                          , batch_size = batch_size
                          , num_workers = num_workers
                          , sampler = train_sampler )
dev_loader = DataLoader( dataset = dataset
                          , batch_size = batch_size
                          , num_workers = num_workers
                          , sampler = dev_sampler )
test_loader = DataLoader( dataset = dataset
                          , batch_size = batch_size
                          , num_workers = num_workers
                          , sampler = test_sampler )

In [ ]:
file, image, label = dataset[397]
toPILImageTransform = transforms.ToPILImage()
image = toPILImageTransform(image)
plt.imshow(image)


In [ ]:
file, image, label = dataset[397]
toPILImageTransform = transforms.ToPILImage()
image = toPILImageTransform(image)
plt.imshow(image)
print(label)

In [ ]:
# Load model
model = CNN_V2_BW()
if (USE_GPU):
    model.cuda()
    
lr = 5e-4
optimizer = optim.Adam(model.parameters(), lr = lr)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
train_loss_arr = []
train_acc_arr = []
train_tpr_arr = []
train_tnr_arr = []

dev_loss_arr = []
dev_acc_arr = []
dev_tpr_arr = []
dev_tnr_arr = []

# Loop over the dataset multiple times
total_num_epochs = total_epochs + num_epochs
for epoch in range(num_epochs):
    curr_epoch = total_epochs + epoch + 1
    # Keep track of training loss
    train_loss = []
    # Keep track of dev loss
    dev_loss = []
    # Keep track of accuracy measurements
    acc, tpr, tnr = 0.0, 0.0, 0.0

    # Train the model
    start_time = time.time()
    model.train()
    for batch_idx, (file, image, label) in enumerate(train_loader):
        if USE_GPU:
            data, target = image.cuda(), label.cuda()
        else:
            data, target = image, label
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward pass
        output = model(data)
        # Update target to be the same dimensions as output
        target = target.view(output.shape[0], 1).float()
        # Get accuracy measurements
        acc, tpr, tnr = training_accuracy(output, target, batch_idx, acc, tpr, tnr)
        # Calculate the batch's loss
        curr_train_loss = criterion(output, target)
        # Update the training loss
        train_loss.append(curr_train_loss.item())
        # Backward pass
        curr_train_loss.backward()
        # Perform a single optimization step to update parameters
        optimizer.step()
        # Print debug info every 32 batches
        if (batch_idx) % 32 == 0:
            print('Epoch {}/{}; Iter {}/{}; Loss: {:.4f}; Acc: {:.3f}; True Pos: {:.3f}; True Neg: {:.3f}'
                   .format(curr_epoch, total_num_epochs, batch_idx + 1, len(train_loader), curr_train_loss.item(), acc, tpr, tnr))
    end_time = time.time()
    
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        for batch_idx, (file, image, label) in enumerate(dev_loader):
            if USE_GPU:
                data, target = image.cuda(), label.cuda()
            else:
                data, target = image, label
            # Get predicted output
            output = model(data)
            # Update target to be the same dimensions as output
            target = target.view(output.shape[0], 1).float()
            # Get accuracy measurements
            dev_acc, dev_tpr, dev_tnr = dev_accuracy(output, target)
            # Calculate the batch's loss
            curr_dev_loss = criterion(output, target)
            # Update the dev loss
            dev_loss.append(curr_dev_loss.item())
    
    # Calculate average loss
    avg_train_loss = np.mean(np.array(train_loss))
    avg_dev_loss = np.mean(np.array(dev_loss))
    
    # Update dev loss arrays
    dev_loss_arr.append(avg_dev_loss)
    dev_acc_arr.append(dev_acc)
    dev_tpr_arr.append(dev_tpr)
    dev_tnr_arr.append(dev_tnr)

    # Update training loss arrays
    train_loss_arr.append(avg_train_loss)
    train_acc_arr.append(acc)
    train_tpr_arr.append(tpr)
    train_tnr_arr.append(tnr)

    print('Epoch {}/{}; Avg. Train Loss: {:.4f}; Train Acc: {:.3f}; Train TPR: {:.3f}; Train TNR: {:.3f}; Epoch Time: {} mins; \nAvg. Dev Loss: {:.4f}; Dev Acc: {:.3f}; Dev TPR: {:.3f}; Dev TNR: {:.3f}\n'
        .format(curr_epoch, total_num_epochs, avg_train_loss, acc, tpr, tnr, round((end_time - start_time)/ 60., 2), avg_dev_loss, dev_acc, dev_tpr, dev_tnr))
    
    if avg_dev_loss < dev_loss_min:
        print('Dev loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
              .format(dev_loss_min, avg_dev_loss))
        dev_loss_min = avg_dev_loss
        is_best = False
        if (dev_acc >= dev_acc_max):
            is_best = True
            dev_acc_max = dev_acc
        state = fetch_state( epoch = curr_epoch
                            , model = model
                            , optimizer = optimizer
                            , dev_loss_min = dev_loss_min
                            , dev_acc_max = dev_acc_max )
        save_checkpoint( state = state
                        , is_best = is_best )
        bad_epoch_count = 0
    # If dev loss didn't improve, increase bad_epoch_count and stop if
    # bad_epoch_count >= early_stop_limit
    else:
        bad_epoch_count += 1
        print('{} epochs of increasing dev loss ({:.6f} --> {:.6f}).'
              .format(bad_epoch_count, dev_loss_min, avg_dev_loss))
        if (bad_epoch_count >= early_stop_limit):
            print('Stopping training')
            stop = True

    if (stop):
        break

In [ ]:
#Training set plot
plt.plot(train_loss_arr, color="red")
plt.plot(train_acc_arr, color="blue")
plt.plot(train_tpr_arr, color="green")
plt.plot(train_tnr_arr, color="orange")
plt.show()

In [ ]:
#Dev set plot
plt.plot(dev_loss_arr, color="red")
plt.plot(dev_acc_arr, color="blue")
plt.plot(dev_tpr_arr, color="green")
plt.plot(dev_tnr_arr, color="orange")
plt.show()

In [ ]:
def predict(model, dataloader):
    model.eval()
    acc, tpr, tnr = 0.0, 0.0, 0.0
    y_true, y_hat = [], []
    with torch.no_grad():
        for file, image, label in dataloader:
            if USE_GPU:
                data, target = image.cuda(), label.cuda()
            else:
                data, target = image, label
    
            # Get predicted output
            output = model(data)
            #output = sigmoid(output.cpu()) >= 0.5
            
            # Update target to be the same dimensions as output
            target = target.view(output.shape[0],1).float()
            #target = target >= 0.5
            
            acc, tpr, tnr = training_accuracy(output, target, 1, acc, tpr, tnr)
            #print('File: {}, Output: {}, Target: {}'.format(file, output, target))
            
            y_true.append(target >= 0.5)
            y_hat.append(sigmoid(output.cpu()) >= 0.5)
    return acc, tpr, tnr, y_true, y_hat

def fetch_analysis_data(model, dataloader, totalToReturn):
    model.eval()
    truePositives = []
    trueNegatives = []
    falsePositives = []
    falseNegatives = []
    stopFetch = False
    toPILImageTransform = transforms.ToPILImage()
    with torch.no_grad():
        for file, image, label in dataloader:
            if USE_GPU:
                data, target = image.cuda(), label.cuda()
            else:
                data, target = image, label
            output = model(data)
            output = sigmoid(output.cpu()) >= 0.5  
            target = target.view(output.shape[0],1).float()
            target = target.cpu() >= 0.5
            for i in range(len(output)):
                y_true = target[i]
                y_hat = output[i]
                sample = { 
                    'file' : file[i]
                    , 'image' : toPILImageTransform(image[i])
                    , 'y_true' : y_true.numpy()
                    , 'y_hat' : y_hat.numpy()
                }
                if ( (len(truePositives) >= totalToReturn)
                    and (len(falseNegatives) >= totalToReturn)
                    and (len(trueNegatives) >= totalToReturn)
                    and (len(falsePositives) >= totalToReturn) ):
                    stopFetch = True
                    break
            
                if (y_true == 1):
                    if (y_hat == y_true):
                        if (len(truePositives) >= totalToReturn):
                            continue
                        else:
                            truePositives.append(sample)
                    else:
                        if (len(falseNegatives) >= totalToReturn):
                            continue
                        else:
                            falseNegatives.append(sample)
                else:
                    if (y_hat == y_true):
                        if (len(trueNegatives) >= totalToReturn):
                            continue
                        else:
                            trueNegatives.append(sample)
                    else:
                        if (len(falsePositives) >= totalToReturn):
                            continue
                        else:
                            falsePositives.append(sample)
            if (stopFetch):
                break
    return truePositives, trueNegatives, falsePositives, falseNegatives

In [ ]:
for i in range(10):
    checkpoint = load_checkpoint(model = model, optimizer = None, checkpoint = best_checkpoint)

    # Get accuracy
    acc_t, tpr_t, tnr_t, y_true_t, y_hat_t = predict(test_model, test_loader)
    print('Acc: {}, TPR: {}, TNR: {}'.format(acc_t, tpr_t, tnr_t))

In [ ]:
for i in range(10):
    checkpoint = load_checkpoint(model = model, optimizer = None, checkpoint = best_checkpoint)

    # Get accuracy
    acc_t, tpr_t, tnr_t, y_true_t, y_hat_t = predict(test_model, dev_loader)
    print('Acc: {}, TPR: {}, TNR: {}'.format(acc_t, tpr_t, tnr_t))